In [71]:
import json
import requests
import numpy as np
import pandas as pd
import time
import datetime

In [2]:
pd.set_option('display.max_rows', 20)

In [3]:
lol_api_key = 'RGAPI-bd616a4d-342b-44bb-bd7b-18885aa94a4b'

In [4]:
URLS = {'challenge' : 'https://kr.api.riotgames.com/lol/league/v3/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=',
        'master' : 'https://kr.api.riotgames.com/lol/league/v3/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=',
        'summoners' : 'https://kr.api.riotgames.com/lol/summoner/v3/summoners/',
        'matches' : 'https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/',
        'match_detail' : 'https://kr.api.riotgames.com/lol/match/v3/matches/',
        'champions': 'https://kr.api.riotgames.com/lol/static-data/v3/champions?locale=ko_KR&dataById=false&api_key='
       }

# Data Target
- 특정 게임 버전에서 (7.21) challenge league player가 한명이라도 속한 경기.
- Note: https://docs.google.com/spreadsheets/d/1kd6EdIA9FRWOoRbpY_htll4ijSPyrKc9fOhBYIEViK0/edit#gid=853537432

# Ben & pick order check

같은 팀 벤 중복은 불가능  
  
streamerName = '해물파전'  
1.  
accountId: 2663347  
gameId: 2994667322  
https://www.youtube.com/watch?v=qs43TtFqwRk&t=394s

2.  
accountId: 2663347  
gameId: 2994630626  
https://www.youtube.com/watch?v=y62J44JccSU&t=9s  



In [5]:
url = requests.get(URLS['champions'] + lol_api_key)
print(url.status_code)
champions_info = url.json()

200


In [6]:
df_champions = pd.DataFrame()
for name in champions_info['data'].keys():
    df_champions = df_champions.append([[champions_info['data'][name]['key'], champions_info['data'][name]['id']]])
df_champions.reset_index(inplace=True)
del df_champions['index']
df_champions.columns = ['key','id']

In [7]:
df_champions.tail()

,key,id
133,Shen,98
134,JarvanIV,59
135,Malzahar,90
136,Zac,154
137,Gragas,79


# Data collection

### Summoner target

In [8]:
# challenge league 선수들 정보 불러옴
url = requests.get(URLS['challenge']+lol_api_key)
print(url.status_code)
challenge = url.json()

200


In [9]:
# master league 선수들 정보 불러옴
url = requests.get(URLS['master']+lol_api_key)
print(url.status_code)
master = url.json()

200


In [10]:
# match를 모을 target을 challenge, master 리그 summerner들로 잡음 / 일단 challenge league선수만으로 한다.
df_target_origin = pd.DataFrame(challenge['entries']).append(pd.DataFrame(master['entries']), ignore_index=True)
# df_target_origin = pd.DataFrame(challenge['entries'], ignore_index=True)
df_target_origin[df_target_origin.playerOrTeamName == 'Afreeca Kuro']

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins
126,False,False,False,645,344,1222794,Afreeca Kuro,I,True,413


In [11]:
# X-App-Rate-Limit에 맞춰 호출함. 
# status_code=429 는 api호출 한계 초과임
#     "X-App-Rate-Limit": "100:120,20:1",
#     "X-Method-Rate-Limit": "2000:60",
summoners = []
# counts = 0
for i in df_target_origin.playerOrTeamId:
    time.sleep(0.8)
#     counts += 1
#     print(counts)
    url = requests.get(URLS['summoners'] + i + '?api_key=' + lol_api_key)
    while url.status_code == 429:
        time.sleep(0.8)
        url = requests.get(URLS['summoners'] + i + '?api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
    else:
        summoners.append(url.json())

pd.DataFrame(summoners).to_csv('summonors.csv', index=False)

In [12]:
df_summonors = pd.read_csv('summonors.csv')[['accountId','id']]

In [13]:
# 이렇게 안하면 merge시 조인이 안됨 <- 데이터 타입이 달라서 생기는 문제인데 꼭 이렇게 해야 할까?
df_target_origin['playerOrTeamId'] = df_target_origin['playerOrTeamId'].astype(int)

In [14]:
df_target = pd.merge(df_target_origin, df_summonors, left_on='playerOrTeamId', right_on = 'id', how='inner')

In [15]:
df_target[df_target.playerOrTeamName == 'Afreeca Kuro']

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins,accountId,id
126,False,False,False,645,344,1222794,Afreeca Kuro,I,True,413,232934,1222794


In [16]:
df_target.to_csv('target.csv', index=False)

### Match

In [111]:
df_target = pd.read_csv('target.csv')
df_target.tail()

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins,accountId,id
804,False,True,False,63,415,32094586,sen Haksal,I,False,475,205446910,32094586
805,False,True,False,58,184,9143363,강남1번원딜,I,False,233,6274737,9143363
806,False,False,False,87,1017,2029599,XuanXuanPi,I,True,1060,1543417,2029599
807,True,True,False,103,821,10393044,시공 여행자,I,False,894,7256651,10393044
808,False,False,False,102,109,39180949,섹지드,I,False,160,206559116,39180949


In [22]:
# version별 분석을 하고 싶어서 기준일을 timestamp로 지정한다. + 9 시간을 고려한다.
# 7.21 시작일 2017-10-25 한국시간기준
timestamp_begin = int(datetime.datetime.timestamp(datetime.datetime(2017,10,26,0,0,0))) * 1000
timestamp_end = int(datetime.datetime.timestamp(datetime.datetime(2017,10,31,0,0,0))) * 1000

print(timestamp_begin, timestamp_end)

1508943600000 1509375600000


In [41]:
# match 를 선수별로 모은다.
# 선수당 max 100개만 모은다. 라이엇 정책이 1회 조회에 100개. / 100개 이상으로도 돌릴 수 있지만 이번엔 이정도까지만.
# "X-Method-Rate-Limit-Count": "1:10"
# "X-App-Rate-Limit": "100:120,20:1"
# "status_code": 404, "message": "Not found" 
# - challenge league summoner인데 이번 버전에서 플레이 하지 않아서 찾을 수 있는 값이 없을 수 있음.

matches = dict()
accountId_exeptions = []
counts = 0
for i in df_target.accountId:
    time.sleep(0.8)
#     counts += 1
#     print(counts, i)
    url = requests.get(URLS['matches'] + str(i) + '?endTime=' + str(timestamp_end) + '&beginTime=' + \
                       str(timestamp_begin) + '&api_key=' + lol_api_key)
    while url.status_code == 429:
        time.sleep(0.8)
        url = requests.get(URLS['matches'] + str(i) + '?endTime=' + str(timestamp_end) + '&beginTime=' + \
                           str(timestamp_begin) + '&api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
        accountId_exeptions.append([i, url.status_code])
    else:
        matches[str(i)] = url.json()

with open('matches.json', 'w') as js:
     json.dump(matches, js)

1 209050712
2 206737570
3 208954564
4 665129
665129 404
5 200578193
6 208710504
7 205721952
8 4172890
9 201009223
10 6728662
11 1437517
12 4094465
13 203024256
14 1646013
15 201556024
16 207874718
17 203509833
18 2725181
19 205817334
20 1805320
21 208177961
22 209270513
23 5524615
24 208543788
25 200555547
26 4122399
27 208259196
28 206325287
29 205888262
30 779882
31 377501
32 200030188
33 200059718
34 2024821
35 266599
36 204912588
37 204152129
38 204062750
39 206298836
40 203874706
41 203649218
42 208056637
43 1830884
44 200381865
45 205248873
46 206107682
47 3489417
48 7803418
49 909429
50 1960910
51 4020503
52 207987339
53 3765312
54 208537462
55 206198425
56 208244782
57 209521897
58 205940681
59 204167188
60 208191209
61 4741330
62 201236882
63 6871165
6871165 404
64 203073673
65 4267501
4267501 404
66 209319733
67 209084978
68 208484750
69 206560208
70 205005943
71 3972036
72 207569128
73 204500813
74 755064
75 206227817
76 209394196
77 5775781
78 7751954
79 209676025
80 429906

596 1816861
597 208279320
598 204802627
204802627 404
599 209197502
600 207155639
601 2735812
602 204531622
204531622 404
603 930698
604 209334636
605 1302978
606 201320486
607 2654706
608 208162181
609 207653439
610 2373709
611 4307493
612 201711371
613 7579092
614 7158870
615 3095385
616 4006088
617 201895625
618 200539060
619 5733995
620 1824284
621 209012031
622 205213320
205213320 404
623 206431508
624 200659835
625 4495828
626 2355618
627 209192190
628 6554107
629 202047579
630 7555939
631 4939473
632 209217739
633 206180514
634 5996803
635 3414924
636 205043205
637 208306070
638 208436226
639 202575872
640 6286773
641 3632481
642 201599124
643 7859270
644 208660746
645 201364900
646 205480506
647 200050008
648 203735788
649 3752918
650 2136898
651 5046059
652 206101124
653 208917246
654 1783885
655 204326691
656 203359828
657 202567851
658 209306798
659 206051565
660 6655923
661 208627998
662 209106299
663 208168133
208168133 404
664 1073266
1073266 404
665 1755389
666 7607727
6

### gameId

- challenge league summoner가 한명이라도 속한 경기를 추출한다.

In [42]:
with open('matches.json', 'r') as js:
     mataches_json = json.load(js)

In [47]:
mataches_json['7256651'].keys()

dict_keys(['matches', 'startIndex', 'endIndex', 'totalGames'])

In [48]:
gameIds = []
for accountId in mataches_json:
    for i in range(mataches_json[accountId]['endIndex']):
        gameIds.append(mataches_json[accountId]['matches'][i]['gameId'])
unique_gameIds = list(set(gameIds))

### gameId_detail

In [83]:
# match 세부사항을 gameId별로 모은다.
# "X-App-Rate-Limit": "100:120,20:1"
# "X-Method-Rate-Limit": "500:10"
# len(unique_gameIds) = 5984
match_details = dict()
gameID_exeptions = []
counts = 0
for unique_gameId in unique_gameIds:
    time.sleep(0.4)
    counts += 1
    print(counts, unique_gameId)
    url = requests.get(URLS['match_detail'] + str(unique_gameId) + '?api_key=' + lol_api_key)
    while url.status_code == 429:
        time.sleep(0.8)
        url = requests.get(URLS['match_detail'] + str(unique_gameId) + '?api_key=' + lol_api_key)
    if url.status_code != 200:
        print(unique_gameId, url.status_code)
        gameID_exeptions.append([unique_gameId, url.status_code])
    else:
        match_details[str(unique_gameId)] = url.json()

# with open('match_details.json', 'w') as js:
#      json.dump(match_details, js)

1 2997580094
2 2997547345
3 2993353049
4 2993254752
5 2996171106
6 2994663786
7 2993254773
8 2994205045
9 2995646843
10 2994106749
11 2993942929
12 2994565525
13 2997514650
14 2994467233
15 2995319201
16 2996564387
17 2993975719
18 2995614121
19 2995155376
20 2994139572
21 2993156535
22 2993320376
23 2993516985
24 2995122620
25 2995483071
26 2997481920
27 2994598353
28 2996531666
29 2995974621
30 2995319279
31 2995188218
32 2995974652
33 2996171261
34 2993320450
35 2996433422
36 2993320463
37 2994270742
38 2994106907
39 2996269606
40 2994401831
41 2994401838
42 2995155503
43 2994303536
44 2994663986
45 2997416505
46 2997252681
47 2997711439
48 2995057235
49 2996367956
50 2997088853
51 2995647061
52 2997482074
53 2996466272
54 2994303585
55 2996335227
56 2994565760
57 2996368020
58 2995843751
59 2996826796
60 2996859564
61 2997252788
62 2994238136
63 2994205381
64 2994664137
65 2997514953
66 2996368075
67 2994533079
68 2995483352
69 2997383897
70 2996826842
71 2997613277
72 2994565855
7

555 2995682595
556 2994634020
557 2994273577
558 2993487145
559 2996403497
560 2994109744
561 2996370736
562 2996763961
563 2993519934
564 2993880386
565 2993192258
566 2994076996
567 2994404691
568 2994208085
569 2996960605
570 2994109792
571 2995158375
572 2994666856
573 2994470262
574 2997648761
575 2997124478
576 2997550470
577 2997616017
578 2995125650
579 2993225105
580 2996272532
581 2996567451
582 2996436381
583 2994503069
584 2994372012
585 2994503088
586 2996436401
587 2994109873
588 2993520052
589 2997583287
590 2993880503
591 2993520057
592 2997714362
593 2996567489
594 2995617223
595 2996370892
596 2996633043
597 2995060179
598 2997616106
599 2994404846
600 2994142703
601 2995387890
602 2993225210
603 2996272637
604 2993257992
605 2993356301
606 2994372112
607 2996993574
608 2995224115
609 2993290813
610 2994601535
611 2993192516
612 2993192522
613 2996633163
614 2995715660
615 2995519053
616 2993389134
617 2994896465
618 2996829789
619 2994208361
620 2993454700
621 299414

### 1차 돌리기: 오류나기 전까지

In [57]:
# count:5301 gameID: 2994336055 까지임
match_details1 = match_details.copy()
len(list(match_details1.keys()))

In [59]:
with open('match_details1.json', 'w') as js:
     json.dump(match_details1, js)

### 2차 돌리기

In [84]:
# count:5302 gameID: 2997580094 부터임
match_details2 = match_details.copy()
len(list(match_details2.keys()))

In [86]:
with open('match_details2.json', 'w') as js:
     json.dump(match_details2, js)

### 여러개 match_details 합치기

In [106]:
# match_details1 ,2 합치기
match_details_total = match_details1.copy()
match_details_total.update(match_details2)
len(match_details_total)

5985

In [107]:
with open('match_details_total.json', 'w') as js:
     json.dump(match_details_total, js)

# 참고자료

In [ ]:
API_KEY = os.environ['LOL_API_KEY']
URLS = {
        'NAME_TO_ID' : 'https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-name/{name}',
        'ID_TO_NAME' : 'https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-account/{id}',
        'MATCHES_BY_ID' : 'https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/{id}?season=8',
        'RECENT_MATCHES_BY_ID' : 'https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/{id}/recent',
        'MATCH_INFO': 'https://na1.api.riotgames.com/lol/match/v3/matches/{id}', 
        'RECENT_TEN' : 'https://na.api.riotgames.com/api/lol/NA/v1.3/game/by-summoner/{summonerId}/recent',
        }
HEADERS = {'X-Riot-Token' : API_KEY}

CHAMP_BY_ID = {}
with open('data/champ_by_id.json') as f:
    CHAMP_BY_ID = json.load(f)

CHAMP_BY_NAME = {}
with open('data/champ_by_name.json') as f:
    CHAMP_BY_NAME = json.load(f)


print(CHAMP_BY_ID['data']['236'])

In [ ]:
from apiKeys import API_KEY
import requests
import sys
import pprint

USERNAME = sys.argv[1]
BASE_URL = "https://na1.api.riotgames.com/lol/"
SUMMONER_ID_URL = BASE_URL + "summoner/v3/summoners/by-name/"
MASTERY_URL = BASE_URL + "platform/v3/masteries/by-summoner/"

r = requests.get(SUMMONER_ID_URL + USERNAME + "?api_key=" + API_KEY)
print r.status_code
data = r.json()
print data
print data["name"]
print data["id"]
summoner_id = data["id"]
r = requests.get(MASTERY_URL + str(summoner_id) + "?api_key=" + API_KEY)
pprint.pprint(r.json())